Arquivo de análise usando a frequência: analise_doc_term_matriz_freq_with_header_discrete_AND.txt
Arquivo de análise usando tf-idf: analise_doc_term_matriz_tfidf_with_header_discrete_AND.txt

#### Sequência de leitura do arquivo
1- Nome do arquivo que gerei para a análise<br />
2- Nome do arquivo que o professor gerou para a análise<br />
3- Nome do arquivo base usado para gerar os grupos<br />
4- Número total Colunas (atributos)<br />
5- Número total de Linhas (exemplos/amostras)<br />
6- Nome da métrica de avaliação<br />
7- Operador entre características usado pelo SSDP<br />
##### // A partir daqui se repete até a última classe //
8- Quebra de linha<br />
9- Número identificador da classe avaliada<br />
10- Quebra de linha<br />
11- O conjunto de regras gerado a partir do PageRank<br />
12- Quebra de linha<br />
13- O conjunto de regras gerado a partir do total<br />

In [12]:
import pandas as pd
import numpy as np

In [15]:
headerList = []
# {ID: {'rulesA': [], 'rulesB': [], 'intersection': [], 'somenteEmA': [], 'somenteEmB': []}}
# dfCommunity = {}
dfCommunity = pd.DataFrame()#columns=['rulesA', 'rulesB', 'intersection', 'somenteEmA', 'somenteEmB'])
# dfCommunity = pd.DataFrame()

def colectDataFromFile(fileName):
    readHeader = False
    dfRulesA = pd.DataFrame(columns=['community', 'rules'])
    dfRulesB = pd.DataFrame(columns=['community', 'rules'])

    with open('../testingPageRank/' + fileName) as f:
        lastID = ''
        countSets = 0

        for line in f:
            cleanLine = line.replace('\n','').strip()

            if not readHeader:
                headerList.append(cleanLine)
            else:
                if cleanLine.isdigit():
                    lastID = cleanLine

                elif '{' in cleanLine:
                    cleanLine = cleanLine[1:-1]

                    if countSets < 20:
                        dfRulesA.loc[len(dfRulesA)] = cleanLine
                    else:
                        dfRulesB.loc[len(dfRulesB)] = cleanLine

                    countSets += 1

                if countSets > 0 and len(dfRulesB) == 20:
                    # interseção
                    dfIntersection = pd.merge(dfRulesA, dfRulesB, how='inner', on=['rules'])
                    dfIntersection.drop(['community_x', 'community_y'], axis=1, inplace=True)
                    dfIntersection.columns = ['rulesIntersection']
                    
                    # as diferenças entre os dataFrames
                    setA = set(dfRulesA['rules'])
                    setB = set(dfRulesB['rules'])
                    
                    print("Tamanho de A-B:", len(list(setA - setB)))
                    
                    listNan = np.empty(1, len(dfRulesA['rules']) - len(list(setA - setB)))
                    listNan = np.nan
                    
                    dfRulesA.columns = ['community', 'rulesA']
                    dfRulesB.columns = ['community', 'rulesB']
                    dfRulesA['community'] = lastID
                    dfRulesB['community'] = lastID
                    
                    dfCommunity = pd.concat([dfRulesA, dfRulesB.drop(['community'], axis=1), dfIntersection], axis=1)
                    
                    dfCommunity['onlyOnA'] = list(setA - setB)
                    dfCommunity['onlyOnB'] = list(setB - setA)
                    
                    dfRulesA = pd.DataFrame(columns=['community', 'rules'])
                    dfRulesB = pd.DataFrame(columns=['community', 'rules'])
    
                    countSets = 0

                    dfCommunity.to_csv('filesResults/rules_community_' + lastID + '.csv', header=True, index=False,
                                       sep=';')
            
                    dfCommunity = pd.DataFrame()

            if cleanLine == 'AND' or cleanLine == 'OR':
                readHeader = True

#     print(dfCommunity['6']['rulesA'])
#     print(len(dfCommunity['6']['rulesA']))
#     print(len(dfCommunity['6']['rulesB']))
#     print('Intersection:', dfCommunity['256']['intersection'])
#     print()
#     print('Somente em B:', dfCommunity['256']['somenteEmB'])
#     print('Size somente em B:', len(dfCommunity['256']['somenteEmB']))

##### Testing data collect

In [16]:
colectDataFromFile('analise_doc_term_matriz_freq_with_header_discrete_AND.txt')
# colectDataFromFile('analise_doc_term_matriz_tfidf_with_header_discrete_AND.txt')

Tamanho de A-B: 17


TypeError: data type not understood

,rulesA,rulesB,intersection,somenteEmA,somenteEmB


<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 5 columns):
rulesA          0 non-null object
rulesB          0 non-null object
intersection    0 non-null object
somenteEmA      0 non-null object
somenteEmB      0 non-null object
dtypes: object(5)
memory usage: 0.0+ bytes
